In [ ]:
import numpy as np
import tensorflow.keras as keras
import tensorflow as tf

inputs = keras.Input((6))
x = keras.layers.Dense(40, activation="selu")(inputs)
x = keras.layers.Dense(20, activation="selu")(x)
outputs = keras.layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(loss=keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam())

intermediate_model = keras.Model(inputs, x)

In [ ]:
import tensorflow_datasets as tfds
from sklearn import datasets

x, y = datasets.fetch_california_housing(return_X_y=True)

x = x[:, :6]

base_x, base_y = x.copy(), y.copy()

x_train, y_train, x_test, y_test = x[:10000], y[:10000], x[10000:], y[10000:]

base_x_train, base_y_train, base_x_test, base_y_test = x_train.copy(), y_train.copy(), x_test.copy(), y_test.copy()

In [ ]:

# VERY coarse normalisation. Don't really care that much about performance on validation set
# but still need to scale the data
ZERO_MAX = x_train[:, 0].max()
x_test[:, 0] /= ZERO_MAX
x_train[:, 0] /= ZERO_MAX

ONE_MAX = x_train[:, 1].max()

x_test[:, 1] /= ONE_MAX
x_train[:, 1] /= ONE_MAX

x_test[:, 2] = np.clip(x_train[:, 2], 0, 20) / 20
x_train[:, 2] = np.clip(x_train[:, 2], 0, 20) / 20

x_test[:, 3] = np.clip(x_train[:, 3], 0, 3) / 3
x_train[:, 3] = np.clip(x_train[:, 3], 0, 3) / 3

x_test[:, 4] = np.log10(x_train[:, 4]) - 3
x_train[:, 4] = np.log10(x_train[:, 4]) - 3

x_test[:, 5] = np.clip(x_train[:, 5], 0, 8) / 8
x_train[:, 5] = np.clip(x_train[:, 5], 0, 8) / 8

# SIX_MAX = x_train[:, 6].max()

# x_test[:, 6] = (x_test[:, 6] - 30) / SIX_MAX
# x_train[:, 6] = (x_train[:, 6] - 30) / SIX_MAX

# SEVEN_MIN = np.abs(x_train[:, 7].min())

# x_test[:, 7] = (x_test[:, 7] + SEVEN_MIN) / 10
# x_train[:, 7] = (x_train[:, 7] + SEVEN_MIN) / 10

In [ ]:
def reverse_encoding(case):
    case = case.copy()
    case[0] *= ZERO_MAX
    case[1] *= ONE_MAX
    case[2] *= 20
    case[3] *= 3
    case[4] = 10 ** (case[4] + 3)
    case[5] *= 8
    # case[6] = case[6] * SIX_MAX + 30
    # case[7] = case[7] * 10 - SEVEN_MIN

    return case

In [ ]:
model.fit(x_train, y_train, epochs=12, validation_data=(x_test, y_test), batch_size=64)

In [ ]:
import seaborn as sb
sb.histplot(x_test[:, 3] / x_test[:, 5])

In [ ]:
ratio = x_test[:, 3] / x_test[:, 5]

concept_index = ratio

In [ ]:
probe_input = keras.Input(20)
probe_output = keras.layers.Dense(1, activity_regularizer=keras.regularizers.L1(0.01), activation="relu")(probe_input)

probe = keras.Model(probe_input, probe_output)
probe.compile(loss=keras.losses.MeanAbsoluteError(), optimizer=keras.optimizers.Adam())

inter_x_test = intermediate_model(x_test)
probe_x_train, probe_y_train, probe_x_test, probe_y_test = inter_x_test[:2000], concept_index[:2000], inter_x_test[2000:], concept_index[2000:]

In [ ]:
probe.fit(probe_x_train, probe_y_train, validation_data=(probe_x_test, probe_y_test), epochs=17)

In [ ]:
main_inputs = keras.Input(6)
maximiser_input = keras.Input(1)
max_plane = keras.layers.Dense(6, name="max_plane", activity_regularizer=keras.regularizers.L1(0.25), activation="linear")(maximiser_input)
additive = keras.layers.Add()([max_plane, main_inputs])

intermediate_model.trainable = False
inter_outputs = intermediate_model(additive)

maximised_probe_output = keras.layers.Dense(1, name="max_probe_repl", activation="relu")(inter_outputs)
maximiser_model = keras.Model([main_inputs, maximiser_input], maximised_probe_output)
look_at_results_model = keras.Model([main_inputs, maximiser_input], additive)
max_plane_model = keras.Model([main_inputs, maximiser_input], max_plane)

maximiser_model.get_layer("max_probe_repl").set_weights(probe.layers[-1].get_weights())
maximiser_model.get_layer("max_probe_repl").trainable = False
maximiser_model.get_layer("max_plane").trainable = True

maximiser_model.compile(loss=keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam())

In [ ]:
np.argpartition(concept_index, 5)[:5]

In [ ]:
CASE_INDEX = 10248

In [ ]:
concept_index[CASE_INDEX]

In [ ]:
case = np.repeat(np.expand_dims(x_test[CASE_INDEX], axis=0), 64, axis=0)
ones = np.expand_dims(np.ones(64), -1)

In [ ]:
maximiser_model.fit([case, ones], ones, epochs=300, batch_size=32)

In [ ]:
maximiser_model([case, ones])[0].numpy()

In [ ]:
maximised = look_at_results_model([case, ones])[0].numpy()
reverse_encoding(maximised)


In [ ]:
base_x_test[CASE_INDEX]

In [ ]:
base_x_test[2649, 3] / base_x_test[2649, 5]

In [ ]:
reverse_encoding(maximised)[3] / reverse_encoding(maximised)[5]

In [ ]:
import os

os.makedirs("housing_results", exist_ok=True)

np.savez("housing_results/{}.npz".format(CASE_INDEX), original_case=base_x_test[CASE_INDEX], maximised_case=reverse_encoding(maximised))

In [ ]:
data = np.load("housing_results/{}.npz".format(CASE_INDEX))

In [ ]:
data["original_case"]

In [ ]:
data["maximised_case"]